In [1]:
import pandas as pd

In [2]:
df = pd.read_csv ('../data/michelin_reviews.csv')

In [3]:
df

,reviews
0,Chef Barry Dindyal is the talent behind this p...
1,It may be the signature cry for falling object...
2,Once a pop-up and now a full-fledged brick-and...
3,Chef Rob Sonderman of the Federalist Pig has e...
4,This cozy Ethiopian den is an homage to Owner ...
...,...
1375,The architecturally intriguing Hayden Tract di...
1376,"Set amidst the luxury shops of Rodeo Drive, th..."
1377,"In contrast to its grand hotel setting, this b..."
1378,"Every quarter of the year, Maude finds its new..."


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer(max_features=10_000,stop_words="english")
features = vec.fit_transform(df["reviews"])

print(features.shape) # (11314, 10000)

(1380, 8996)


In [82]:
from sklearn.neighbors import NearestNeighbors
knn = NearestNeighbors(n_neighbors=10, metric='cosine',n_jobs=-1)
knn.fit(features)

NearestNeighbors(metric='cosine', n_jobs=-1, n_neighbors=10)

In [83]:
philly_df = pd.read_csv ('../data/philly_review_p.csv')

In [84]:
philly_urls = pd.read_csv('../data/philly_infatuation_review_urls.csv')
philly_urls

,url
0,/philadelphia/reviews/spice-finch
1,/philadelphia/reviews/bookers-restaurant-bar
2,/philadelphia/reviews/jims-south-st
3,/philadelphia/reviews/fork
4,/philadelphia/reviews/laser-wolf
...,...
570,/philadelphia/reviews/burgs-hideaway-lounge
571,/philadelphia/reviews/pumpkin-restaurant
572,/philadelphia/reviews/day-by-day
573,/philadelphia/reviews/sams-morning-glory-diner


In [85]:
merged_df = philly_df.merge(philly_urls, left_index=True, right_index=True)

In [86]:
input_texts = merged_df['reviews']
input_features = vec.transform(input_texts)

distance = []
neighbor = []

D, N = knn.kneighbors(input_features, n_neighbors=1, return_distance=True)

for input_text, distances, neighbors in zip(input_texts, D, N):
    for dist, neighbor_idx in zip(distances, neighbors):
        distance.append(dist)
        neighbor.append(neighbor_idx)

In [87]:
distance

[0.8821388600948966,
 0.8637985170275357,
 0.8446290754206189,
 0.8527877399772141,
 0.8328906063675073,
 0.8625438922090206,
 0.8713497687709282,
 0.8690861137487058,
 0.8589133666156538,
 0.8577079337292879,
 0.8033203380609222,
 0.7910374239988398,
 0.7887011072366416,
 0.8465395055205129,
 0.8466961317225014,
 0.8822992250080715,
 0.8541199379999154,
 0.8796801825681723,
 0.871981909572004,
 0.8457598547126463,
 0.8814627116998631,
 0.7908290865369327,
 0.8508148515265436,
 0.8406062766621853,
 0.8689565703829922,
 0.8891265370118233,
 0.8762903201634571,
 0.8734341837027155,
 0.888639745738649,
 0.8497913854777996,
 0.8847263722510842,
 0.8081665476763162,
 0.8813467761864754,
 0.8500101900456434,
 0.8452252363237689,
 0.8762262758728544,
 0.8111920412714311,
 0.8653923947358904,
 0.8009150622017365,
 0.7914435748688431,
 0.8443844602648891,
 0.8522553171766041,
 0.8549618332753688,
 0.865079054622729,
 0.8768420970467153,
 0.8550730285732548,
 0.9044050652443332,
 0.8792033504100

In [88]:
merged_df['distance'] = distance

In [89]:
merged_df['neighbor'] = neighbor

In [90]:
merged_df.sort_values('distance')

,reviews,url,distance,neighbor
300,"For a long time, <a class=""styles_anchor___Zdo...",/philadelphia/reviews/la-viola,0.664996,210
274,Hajimaru is a small BYOB ramen shop on Girard ...,/philadelphia/reviews/ants-pants-cafe,0.675889,418
221,"Pop-up Anchor Light, which operates out of the...",/philadelphia/reviews/le-mandingue,0.689490,254
485,There are a few pho places in the city that ev...,/philadelphia/reviews/hipcityveg,0.722978,913
375,There are two things you need to know about Do...,/philadelphia/reviews/browns-restaurant-ocean-...,0.724965,428
...,...,...,...,...
402,Everyone knows that setting a calendar reminde...,/philadelphia/reviews/kohrs-frozen-custard-the...,0.913905,816
468,Flambo sounds like a really bad remake of Ramb...,/philadelphia/reviews/dock-street-brewery-rest...,0.918992,790
107,[],/philadelphia/reviews/sakana-omakase-sushi,1.000000,0
362,[],/philadelphia/reviews/quahogs-seafood-shack-st...,1.000000,0
